# Aluguel

## Pega URL


In [19]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm

# Função para extrair os URLs dos links <a> com a classe 'new-card'
def extrair_urls_anuncios(soup):
    # Encontra todos os elementos 'a' com a classe 'new-card'
    links = soup.find_all('a', class_='new-card')
    # Extrai os URLs dos links
    urls = [link.get('href') for link in links]
    return urls

# Função para fazer o scraping de uma página e extrair os URLs dos links
def fazer_scraping_pagina(url):
    # Faz a requisição HTTP
    response = requests.get(url)
    # Verifica se a requisição foi bem-sucedida
    if response.status_code == 200:
        # Parseia o conteúdo HTML
        soup = BeautifulSoup(response.text, 'html.parser')
        # Extrai os URLs dos links
        urls_anuncios_pagina = extrair_urls_anuncios(soup)
        return urls_anuncios_pagina
    else:
        print('Falha ao acessar a página:', response.status_code)
        return None

# Função para fazer o scraping de várias páginas
def fazer_scraping_varias_paginas(url_base, num_paginas):
    lista_urls_anuncios_total = []
    for pagina in tqdm(range(1, num_paginas + 1), desc="Progresso"):
        url_pagina = f"{url_base}?page={pagina}"
        urls_anuncios_pagina = fazer_scraping_pagina(url_pagina)
        if urls_anuncios_pagina:
            lista_urls_anuncios_total.extend(urls_anuncios_pagina)
    return lista_urls_anuncios_total

# URL base do site de imóveis que você quer fazer scraping
url_base = 'https://www.dfimoveis.com.br/aluguel/df/todos/imoveis'

# Número total de páginas que você deseja percorrer
num_paginas = 241 # Ajuste conforme necessário

# Faz o scraping de várias páginas
urls_anuncios_total = fazer_scraping_varias_paginas(url_base, num_paginas)

# Se os URLs foram obtidos com sucesso
if urls_anuncios_total:
    # Cria um DataFrame com os URLs dos anúncios
    df_link_aluguel = pd.DataFrame({'URL': urls_anuncios_total})
    # Mostra o DataFrame
    print(df_link_aluguel)


Progresso: 100%|██████████| 241/241 [03:18<00:00,  1.21it/s]

                                                    URL
0     /imovel/prdio-0-quartos-aluguel-asa-sul-brasil...
1     /imovel/hotel-flat-1-quarto-aluguel-asa-sul-br...
2     /imovel/apartamento-2-quartos-aluguel-noroeste...
3     /imovel/sala-0-quartos-aluguel-norte-aguas-cla...
4     /imovel/sala-0-quartos-aluguel-sudoeste-brasil...
...                                                 ...
7225  /imovel/galpo-0-quartos-aluguel-sia-setor-indu...
7226  /imovel/casa-4-quartos-aluguel-lago-sul-brasil...
7227  /imovel/galpo-0-quartos-aluguel-polo-jk-santa-...
7228  /imovel/casa-condominio-4-quartos-aluguel-lago...
7229  /imovel/sala-0-quartos-aluguel-asa-sul-brasili...

[7230 rows x 1 columns]


## Raspagem dos anúncios

In [20]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm

# Função para extrair as informações de um anúncio a partir de um URL
def extrair_info_anuncio(url):
    # Faz a requisição HTTP para o URL do anúncio
    response = requests.get(url)
    # Verifica se a requisição foi bem-sucedida
    if response.status_code == 200:
        # Parseia o conteúdo HTML
        soup = BeautifulSoup(response.text, 'html.parser')
        # Extrai as informações do anúncio
        try:
            titulo = soup.find('h1', class_='mb-0 font-weight-600 mobile-fs-1-5').text.strip()
            preco = soup.find('h6', class_='align-text-left text-normal bb-m mr-6').text.split('\n')[2].replace('.','')
            area = soup.find('h6', class_='text-normal mb-0').text.split()[2].replace('.','').replace(',',".")
            info = soup.find_all("h6", class_='mb-0 text-normal')
            imobiliaria = soup.find('h6', class_='pb-0 mb-0').text
            creci = soup.find('div',class_='col-md-8 mt-2 ml-2').text.split('\n')[2].split()[1]
            tipo = soup.find('div', class_='col-lg-6').text.split()[2]

            possibilidades = [
            'Aquecimento Solar', 'Circuito de TV', 'Gerador de Energia', 'Piscina', 'Sala de Ginástica',
            'Varanda', 'Cozinha Espaçosa', 'Ar Condicionado', 'Cobertura Coletiva', 'Guarita',
            'PlayGround', 'Salão de Festas', 'Projeto de Iluminação', 'Pintura Nova', 'Área de Lazer',
            'Interfone', 'Mobiliado', 'Área de Serviço', 'Brinquedoteca', 'Churrasqueira', 'Cozinha com Armários',
            'Despensa', 'Escritório', 'Gás Canalizado', 'Jardim', 'Lavabo', 'Lazer no Pilotis', 'Poço Artesiano',
            'Portão Eletrônico', 'Quadra Esportiva', 'Sala de Jogos', 'Salão de Jogos', 'Salão Gourmet', 'Sauna',
            'SPA', 'Área de Serviço Coberta', 'Piso em Porcelanato', 'Vista Livre', 'Home Theater', 'Aceita Pet', 'Academia'
            ]

            resultados = {possibilidade: 0 for possibilidade in possibilidades}
            checkbox = soup.find('ul', class_='checkboxes')
            checkbox_text = checkbox.text.strip()
            checkbox_list = checkbox_text.split('\n')

            for item in checkbox_list:
              if item in resultados:
                resultados[item] = 1

            dados_dict = {}
            for i in info:
              item  = i.text.split()
              chave = item[0]
              valor = ' '.join(item[1:])
              if chave in dados_dict:
                dados_dict[chave].append(valor)
              else:
                dados_dict[chave] = [valor]
            try:
              quartos = dados_dict['Quartos:'][0]
            except:
              quartos = 0
            try:
              suites = dados_dict['Suítes:'][0]
            except:
              suites = 0
            try:
              garagem = dados_dict['Garagens:'][0]
            except:
              garagem = 0
            try:
              iptu = dados_dict['IPTU'][0]
            except:
              iptu = '0 SemInformacao'
            try:
              condominio = dados_dict['Condomínio'][0]
            except:
              condominio = '0 SemInformacao'
            try:
              regiao = dados_dict['Cidade:'][0]
            except:
              regiao = '0 SemInformacao'
        except AttributeError as e:
            print(e)
            return None
        # Retorna um dicionário com as informações do anúncio
        return {
            'Título': titulo,
            "Valor": float(preco),
            "area": float(area),
            'Valor_metro': float(preco)/float(area),
            'quartos': quartos,
            'suites': suites,
            'garagem': garagem,
            'iptu': iptu.split()[1].replace('.',''),
            'condominio': condominio.split()[1].replace('.',''),
            'regiao': regiao.split('-')[1].strip(),
            'cidade': regiao,
            'imobiliaria': imobiliaria,
            'creci': creci,
            "operacao": 'aluguel',
            **resultados,
            'tipo': tipo
        }
    else:
        print('Falha ao acessar o anúncio:', response.status_code)
        return None

# Lista de URLs dos anúncios
urls_anuncios = [f"https://www.dfimoveis.com.br/{pagina}" for pagina in df_link_aluguel['URL'].values]

# Lista para armazenar as informações dos anúncios
info_anuncios_total = []

# Extrai as informações de cada anúncio
for url_anuncio in tqdm(urls_anuncios, desc='Extraindo informações dos anúncios'):
    info_anuncio = extrair_info_anuncio(url_anuncio)
    if info_anuncio:
        info_anuncios_total.append(info_anuncio)

# Se as informações foram obtidas com sucesso
if info_anuncios_total:
    # Cria um DataFrame com as informações dos anúncios
    df_aluguel = pd.DataFrame(info_anuncios_total)

df_aluguel

Extraindo informações dos anúncios: 100%|██████████| 7230/7230 [1:04:06<00:00,  1.88it/s]


,Título,Valor,area,Valor_metro,quartos,suites,garagem,iptu,condominio,regiao,...,Salão Gourmet,Sauna,SPA,Área de Serviço Coberta,Piso em Porcelanato,Vista Livre,Home Theater,Aceita Pet,Academia,tipo
0,SCS Quadra 06 Bloco A Lote 194,7000.0,200.0,35.000000,0,0,0,SemInformacao,2000,ASA SUL,...,0,0,0,0,0,0,0,0,0,Prédio
1,SHS Quadra 02 Bloco J,4000.0,55.0,72.727273,1,1,0,SemInformacao,1216,ASA SUL,...,0,1,0,0,0,0,0,0,0,Hotel
2,SQNW 310 Bloco B,5100.0,76.0,67.105263,2,1,2,409,791,NOROESTE,...,0,1,0,0,0,0,0,0,0,Apartamento
3,Rua COPAIBA,1800.0,32.0,56.250000,0,0,1,SemInformacao,400,NORTE,...,0,0,0,0,0,0,0,0,0,Sala
4,SQSW 300,22100.0,260.0,85.000000,0,0,0,SemInformacao,SemInformacao,SUDOESTE,...,0,0,0,0,0,0,0,0,0,Sala
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7225,SIA Trecho 3,140000.0,6224.0,22.493573,0,0,4,SemInformacao,SemInformacao,SIA,...,0,0,0,0,0,0,0,0,0,Galpão
7226,SHIS QL 8,140000.0,1880.0,74.468085,6,6,14,SemInformacao,SemInformacao,LAGO SUL,...,0,1,0,0,0,0,0,0,0,Casa
7227,Polo de Desenv Juscelino Kubitschek Trecho 5 C...,70000.0,3000.0,23.333333,0,0,4,SemInformacao,SemInformacao,PÓLO JK,...,0,0,0,0,0,0,0,0,0,Galpão
7228,SHIS QI 5 Chacará s 45 a 52,200000.0,3600.0,55.555556,5,5,16,1797,SemInformacao,LAGO SUL,...,0,1,0,0,0,0,0,0,0,Casa


In [21]:
df_aluguel

,Título,Valor,area,Valor_metro,quartos,suites,garagem,iptu,condominio,regiao,...,Salão Gourmet,Sauna,SPA,Área de Serviço Coberta,Piso em Porcelanato,Vista Livre,Home Theater,Aceita Pet,Academia,tipo
0,SCS Quadra 06 Bloco A Lote 194,7000.0,200.0,35.000000,0,0,0,SemInformacao,2000,ASA SUL,...,0,0,0,0,0,0,0,0,0,Prédio
1,SHS Quadra 02 Bloco J,4000.0,55.0,72.727273,1,1,0,SemInformacao,1216,ASA SUL,...,0,1,0,0,0,0,0,0,0,Hotel
2,SQNW 310 Bloco B,5100.0,76.0,67.105263,2,1,2,409,791,NOROESTE,...,0,1,0,0,0,0,0,0,0,Apartamento
3,Rua COPAIBA,1800.0,32.0,56.250000,0,0,1,SemInformacao,400,NORTE,...,0,0,0,0,0,0,0,0,0,Sala
4,SQSW 300,22100.0,260.0,85.000000,0,0,0,SemInformacao,SemInformacao,SUDOESTE,...,0,0,0,0,0,0,0,0,0,Sala
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7225,SIA Trecho 3,140000.0,6224.0,22.493573,0,0,4,SemInformacao,SemInformacao,SIA,...,0,0,0,0,0,0,0,0,0,Galpão
7226,SHIS QL 8,140000.0,1880.0,74.468085,6,6,14,SemInformacao,SemInformacao,LAGO SUL,...,0,1,0,0,0,0,0,0,0,Casa
7227,Polo de Desenv Juscelino Kubitschek Trecho 5 C...,70000.0,3000.0,23.333333,0,0,4,SemInformacao,SemInformacao,PÓLO JK,...,0,0,0,0,0,0,0,0,0,Galpão
7228,SHIS QI 5 Chacará s 45 a 52,200000.0,3600.0,55.555556,5,5,16,1797,SemInformacao,LAGO SUL,...,0,1,0,0,0,0,0,0,0,Casa


## Obter Latitude


In [22]:
# from geopy.geocoders import Nominatim
# import pandas as pd
# import time
# from tqdm import tqdm

# def obter_coordenadas(enderecos):
#     # Crie um objeto geolocalizador do Nominatim (OpenStreetMap)
#     geolocator = Nominatim(user_agent="my_geocoder")

#     coordenadas = []

#     # Itera sobre a lista de endereços com a barra de progresso
#     for endereco in tqdm(enderecos, desc="Progresso"):
#         # Tente encontrar a localização do endereço com um tempo limite maior
#         try:
#             location = geolocator.geocode(endereco, timeout=20)
#         except Exception as e:
#             print(f"Erro ao geocodificar o endereço '{endereco}': {e}")
#             location = None

#         # Se a localização for encontrada, adicione as coordenadas de latitude e longitude à lista
#         if location:
#             coordenadas.append((location.latitude, location.longitude))
#         else:
#             coordenadas.append((None, None))

#         # Espera 1 segundo antes de fazer a próxima solicitação
#         time.sleep(1)

#     return coordenadas

# # Lista de endereços
# enderecos = df_aluguel['Título'].values

# # Chame a função para obter as coordenadas
# coordenadas = obter_coordenadas(enderecos)

# # Crie um DataFrame com os resultados
# df_geo = pd.DataFrame(coordenadas, columns=['Latitude', 'Longitude'])


# df_aluguel = pd.concat([df_aluguel,df_geo],axis=1)

# df_aluguel


In [23]:
# from geopy.geocoders import Nominatim
# import pandas as pd
# import time
# from tqdm import tqdm

# def obter_coordenadas(endereco):
#     # Crie um objeto geolocalizador do Nominatim (OpenStreetMap)
#     geolocator = Nominatim(user_agent="my_geocoder")
#     location = geolocator.geocode(endereco,timeout=10)
#     if location:
#       return location.latitude, location.longitude
#     else:
#       return None, None
# # Lista de endereços
# df_geo_na = df_aluguel[df_aluguel['Latitude'].isna() | df_aluguel['Longitude'].isna()]

# for index, row in tqdm(df_geo_na.iterrows(), desc= 'Obter Coordenadas'):
#   endereco = row['Título'].split()[0]
#   latitude, longitude = obter_coordenadas(endereco)
#   df_aluguel.at[index,'Latitude'] = latitude
#   df_aluguel.at[index,'Longitude'] = longitude


In [24]:
# df[(df['Latitude'] > -15) | (df['Longitude'] < -48)]

## Dados Aluguel

In [25]:
df_aluguel.to_csv('aluguel.csv', index=False)

# Venda

## Pega URL

In [26]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm

# Função para extrair os URLs dos links <a> com a classe 'new-card'
def extrair_urls_anuncios(soup):
    # Encontra todos os elementos 'a' com a classe 'new-card'
    links = soup.find_all('a', class_='new-card')
    # Extrai os URLs dos links
    urls = [link.get('href') for link in links]
    return urls

# Função para fazer o scraping de uma página e extrair os URLs dos links
def fazer_scraping_pagina(url):
    # Faz a requisição HTTP
    response = requests.get(url)
    # Verifica se a requisição foi bem-sucedida
    if response.status_code == 200:
        # Parseia o conteúdo HTML
        soup = BeautifulSoup(response.text, 'html.parser')
        # Extrai os URLs dos links
        urls_anuncios_pagina = extrair_urls_anuncios(soup)
        return urls_anuncios_pagina
    else:
        print('Falha ao acessar a página:', response.status_code)
        return None

# Função para fazer o scraping de várias páginas
def fazer_scraping_varias_paginas(url_base, num_paginas):
    lista_urls_anuncios_total = []
    for pagina in tqdm(range(1, num_paginas + 1), desc="Progresso"):
        url_pagina = f"{url_base}?page={pagina}"
        urls_anuncios_pagina = fazer_scraping_pagina(url_pagina)
        if urls_anuncios_pagina:
            lista_urls_anuncios_total.extend(urls_anuncios_pagina)
    return lista_urls_anuncios_total

# URL base do site de imóveis que você quer fazer scraping
url_base = 'https://www.dfimoveis.com.br/venda/df/todos/imoveis'

# Número total de páginas que você deseja percorrer
num_paginas = 1413 # Ajuste conforme necessário

# Faz o scraping de várias páginas
urls_anuncios_total = fazer_scraping_varias_paginas(url_base, num_paginas)

# Se os URLs foram obtidos com sucesso
if urls_anuncios_total:
    # Cria um DataFrame com os URLs dos anúncios
    df_link = pd.DataFrame({'URL': urls_anuncios_total})
    # Mostra o DataFrame
    print(df_link)

Progresso: 100%|██████████| 1413/1413 [16:32<00:00,  1.42it/s] 

                                                     URL
0      /imovel/lancamento-residencial-turmalina-04-su...
1      /imovel/casa-4-quartos-venda-lago-sul-brasilia...
2      /imovel/casa-condominio-4-quartos-venda-jardim...
3      /imovel/apartamento-3-quartos-venda-asa-sul-br...
4      /imovel/lancamento-coberturas-parque-dos-burit...
...                                                  ...
56515  /imovel/casa-4-quartos-venda-lago-sul-brasilia...
56516  /imovel/lancamento-ilha-bela-residence-03-quar...
56517  /imovel/apartamento-3-quartos-venda-asa-sul-br...
56518  /imovel/apartamento-2-quartos-venda-noroeste-b...
56519  /imovel/casa-4-quartos-venda-vicente-pires-vic...

[56520 rows x 1 columns]


In [29]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm

# Função para extrair as informações de um anúncio a partir de um URL
def extrair_info_anuncio(url):
    # Faz a requisição HTTP para o URL do anúncio
    response = requests.get(url)
    # Verifica se a requisição foi bem-sucedida
    if response.status_code == 200:
        # Parseia o conteúdo HTML
        soup = BeautifulSoup(response.text, 'html.parser')
        # Extrai as informações do anúncio
        try:
            titulo = soup.find('h1', class_='mb-0 font-weight-600 fs-1-5').text.strip()
            preco = soup.find('h6', class_='align-text-left text-normal mb-0 mr-6 bb').text.split('\n')[2].replace('.','')
            area = soup.find('h6', class_='text-normal mb-0').text.split()[2].replace('.','').replace(',',".")
            info = soup.find_all("h6", class_='mb-0 text-normal')
            imobiliaria = soup.find('h6', class_='pb-0 mb-0').text
            creci = soup.find('div',class_='col-md-8 mt-2 ml-2').text.split('\n')[2].split()[1]
            tipo = soup.find('div', class_='col-lg-6').text.split()[2]
            possibilidades = [
            'Aquecimento Solar', 'Circuito de TV', 'Gerador de Energia', 'Piscina', 'Sala de Ginástica',
            'Varanda', 'Cozinha Espaçosa', 'Ar Condicionado', 'Cobertura Coletiva', 'Guarita',
            'PlayGround', 'Salão de Festas', 'Projeto de Iluminação', 'Pintura Nova', 'Área de Lazer',
            'Interfone', 'Mobiliado', 'Área de Serviço', 'Brinquedoteca', 'Churrasqueira', 'Cozinha com Armários',
            'Despensa', 'Escritório', 'Gás Canalizado', 'Jardim', 'Lavabo', 'Lazer no Pilotis', 'Poço Artesiano',
            'Portão Eletrônico', 'Quadra Esportiva', 'Sala de Jogos', 'Salão de Jogos', 'Salão Gourmet', 'Sauna',
            'SPA', 'Área de Serviço Coberta', 'Piso em Porcelanato', 'Vista Livre', 'Home Theater', 'Aceita Pet', 'Academia'
            ]

            resultados = {possibilidade: 0 for possibilidade in possibilidades}
            checkbox = soup.find('ul', class_='checkboxes')
            checkbox_text = checkbox.text.strip()
            checkbox_list = checkbox_text.split('\n')

            for item in checkbox_list:
              if item in resultados:
                resultados[item] = 1

            dados_dict = {}
            for i in info:
              item  = i.text.split()
              chave = item[0]
              valor = ' '.join(item[1:])
              if chave in dados_dict:
                dados_dict[chave].append(valor)
              else:
                dados_dict[chave] = [valor]
            try:
              quartos = dados_dict['Quartos:'][0]
            except:
              quartos = 0
            try:
              suites = dados_dict['Suítes:'][0]
            except:
              suites = 0
            try:
              garagem = dados_dict['Garagens:'][0]
            except:
              garagem = 0
            try:
              iptu = dados_dict['IPTU'][0]
            except:
              iptu = '0 SemInformacao'
            try:
              condominio = dados_dict['Condomínio'][0]
            except:
              condominio = '0 SemInformacao'
            try:
              regiao = dados_dict['Cidade:'][0]
            except:
              regiao = '0 SemInformacao'
        except AttributeError as e:
            # print(e)
            return None
        # Retorna um dicionário com as informações do anúncio
        return {
            'Título': titulo,
            "Valor": float(preco),
            "area": float(area),
            'Valor_metro': float(preco)/float(area),
            'quartos': quartos,
            'suites': suites,
            'garagem': garagem,
            'iptu': iptu.split()[1].replace('.',''),
            'condominio': condominio.split()[1].replace('.',''),
            'regiao': regiao.split('-')[1].strip(),
            'cidade': regiao,
            'imobiliaria': imobiliaria,
            'creci': creci,
            "operacao": 'venda',
            **resultados,
            'tipo': tipo
        }
    else:
        print('Falha ao acessar o anúncio:', response.status_code)
        return None

# Lista de URLs dos anúncios
urls_anuncios = [f"https://www.dfimoveis.com.br/{pagina}" for pagina in df_link['URL'].values]

# Lista para armazenar as informações dos anúncios
info_anuncios_total = []

# Extrai as informações de cada anúncio
for url_anuncio in tqdm(urls_anuncios, desc='Extraindo informações dos anúncios'):
    info_anuncio = extrair_info_anuncio(url_anuncio)
    if info_anuncio:
        info_anuncios_total.append(info_anuncio)

# Se as informações foram obtidas com sucesso
if info_anuncios_total:
    # Cria um DataFrame com as informações dos anúncios
    df = pd.DataFrame(info_anuncios_total)

df

Extraindo informações dos anúncios: 100%|██████████| 56520/56520 [5:20:13<00:00,  2.94it/s]   


,Título,Valor,area,Valor_metro,quartos,suites,garagem,iptu,condominio,regiao,...,Salão Gourmet,Sauna,SPA,Área de Serviço Coberta,Piso em Porcelanato,Vista Livre,Home Theater,Aceita Pet,Academia,tipo
0,SMDB Conjunto 15,4900000.0,583.00,8404.802744,4,4,3,SemInformacao,1800,LAGO SUL,...,0,1,0,0,0,0,0,0,0,Casa
1,Condomínio Santa Bárbara Condomínio Santa Bárbara,1290000.0,473.95,2721.806098,5,3,6,1590,SemInformacao,JARDIM BOTANICO,...,0,0,0,0,0,1,0,0,0,Casa
2,SQS 304,2850000.0,177.00,16101.694915,3,1,0,SemInformacao,1310,ASA SUL,...,0,0,0,0,0,1,0,0,0,Apartamento
3,SHIS QI 25,4590000.0,550.00,8345.454545,4,4,2,SemInformacao,SemInformacao,LAGO SUL,...,0,1,0,0,0,0,0,0,0,Casa
4,Quadra 206 - Sagitarius Residencial Resort,3490000.0,282.00,12375.886525,4,4,4,2895,2566,SUL,...,0,1,1,0,0,1,1,0,0,Apartamento
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42385,SMPW Quadra 23 Conjunto 4,2899000.0,715.00,4054.545455,5,4,8,SemInformacao,SemInformacao,PARK WAY,...,0,0,0,0,0,0,0,0,0,Casa
42386,SHIS QL 12,16500000.0,925.00,17837.837838,4,4,4,SemInformacao,SemInformacao,LAGO SUL,...,0,1,0,0,0,0,0,0,0,Casa
42387,SQS 306 Bloco C Asa sul,2500000.0,178.00,14044.943820,3,1,0,SemInformacao,SemInformacao,ASA SUL,...,0,0,0,0,0,1,0,0,0,Apartamento
42388,SQNW 303 Bloco B,1321739.0,77.33,17092.189319,2,1,1,SemInformacao,SemInformacao,NOROESTE,...,0,1,1,0,0,0,0,0,0,Apartamento


In [30]:
df.to_csv('df_vendas.csv', index=False)

# Obter Latitude

In [ ]:
# from geopy.geocoders import Nominatim
# import pandas as pd
# import time
# from tqdm import tqdm

# def obter_coordenadas(enderecos):
#     # Crie um objeto geolocalizador do Nominatim (OpenStreetMap)
#     geolocator = Nominatim(user_agent="my_geocoder")

#     coordenadas = []

#     # Itera sobre a lista de endereços com a barra de progresso
#     for endereco in tqdm(enderecos, desc="Progresso"):
#         # Tente encontrar a localização do endereço com um tempo limite maior
#         try:
#             location = geolocator.geocode(endereco, timeout=20)  # Aumenta o tempo limite para 10 segundos
#         except Exception as e:
#             print(f"Erro ao geocodificar o endereço '{endereco}': {e}")
#             location = None

#         # Se a localização for encontrada, adicione as coordenadas de latitude e longitude à lista
#         if location:
#             coordenadas.append((location.latitude, location.longitude))
#         else:
#             coordenadas.append((None, None))

#         # Espera 1 segundo antes de fazer a próxima solicitação
#         time.sleep(0)

#     return coordenadas

# # Lista de endereços
# enderecos = df['Título'].values

# # Chame a função para obter as coordenadas
# coordenadas = obter_coordenadas(enderecos)

# # Crie um DataFrame com os resultados
# df_geo = pd.DataFrame(coordenadas, columns=['Latitude', 'Longitude'])


# df = pd.concat([df,df_geo],axis=1)


Progresso: 100%|██████████| 57120/57120 [8:21:36<00:00,  1.90it/s]   


In [36]:
df['tipo'].value_counts()

tipo
Casa                 24021
Apartamento          14130
Lote/Terreno/Área     2826
Kitnet                1413
Name: count, dtype: int64

In [ ]:
# from geopy.geocoders import Nominatim
# import pandas as pd
# import time
# from tqdm import tqdm

# def obter_coordenadas(endereco):
#     # Crie um objeto geolocalizador do Nominatim (OpenStreetMap)
#     geolocator = Nominatim(user_agent="my_geocoder")
#     location = geolocator.geocode(endereco,timeout=10)
#     if location:
#       return location.latitude, location.longitude
#     else:
#       return None, None
# # Lista de endereços
# df_geo_na = df[df['Latitude'].isna() | df['Longitude'].isna()]

# for index, row in tqdm(df_geo_na.iterrows(), desc= 'Obter Coordenadas'):
#   endereco = row['Título'].split()[0]
#   latitude, longitude = obter_coordenadas(endereco)
#   df.at[index,'Latitude'] = latitude
#   df.at[index,'Longitude'] = longitude


Obter Coordenadas: 19040it [3:45:55,  1.40it/s]


In [31]:
df.to_csv('df_venda.csv', index=False)

In [32]:
import pandas as pd

In [33]:
df_aluguel = pd.read_csv('aluguel.csv')
df_venda = pd.read_csv('df_venda.csv')

In [37]:
df_aluguel['tipo'].value_counts()

tipo
Casa           2169
Sala           1446
Galpão         1205
Prédio          723
Apartamento     723
Hotel           482
Kitnet          241
Loja            241
Name: count, dtype: int64

In [ ]:
df_venda

,Título,Valor,area,Valor_metro,quartos,suites,garagem,iptu,condominio,regiao,...,SPA,Área de Serviço Coberta,Piso em Porcelanato,Vista Livre,Home Theater,Aceita Pet,Academia,tipo,Latitude,Longitude
0,Condomínio Santa Bárbara Condomínio Santa Bárbara,1290000.0,473.95,2721.806098,5,3,6,1590,SemInformacao,JARDIM BOTANICO,...,0,0,0,1,0,0,0,Casa,-15.961843,-47.821131
1,SQS 304,2850000.0,177.00,16101.694915,3,1,0,SemInformacao,1310,ASA SUL,...,0,0,0,1,0,0,0,Apartamento,-15.806485,-47.896157
2,SHIS QI 25,4590000.0,550.00,8345.454545,4,4,2,SemInformacao,SemInformacao,LAGO SUL,...,0,0,0,0,0,0,0,Casa,-15.843989,-47.836671
3,Quadra 206 - Sagitarius Residencial Resort,3490000.0,282.00,12375.886525,4,4,4,2895,2566,SUL,...,1,0,0,1,1,0,0,Apartamento,-23.299332,-48.054657
4,SQNW 104 Bloco J,2095000.0,141.00,14858.156028,3,3,2,4200,1600,NOROESTE,...,0,0,0,1,0,0,0,Apartamento,-15.758636,-47.911805
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57115,SHIS QI 7,12900000.0,650.00,19846.153846,4,4,4,SemInformacao,SemInformacao,LAGO SUL,...,0,0,0,0,0,0,0,Casa,-15.844132,-47.891195
57116,SHIS QL 10,18500000.0,850.00,21764.705882,5,5,6,SemInformacao,SemInformacao,LAGO SUL,...,0,0,0,0,0,0,0,Casa,-15.829325,-47.872777
57117,SHA Conjunto 5 Chacará 19A,1500000.0,500.00,3000.000000,4,3,3,SemInformacao,SemInformacao,ARNIQUEIRA,...,0,0,0,0,0,0,0,Casa,31.197678,121.334483
57118,Condomínio Santa Felicidade,1340000.0,250.00,5360.000000,3,3,2,SemInformacao,750,JARDIM BOTANICO,...,0,0,0,0,0,0,0,Casa,-15.572722,-47.333241


In [34]:
df_total = pd.concat([df_aluguel,df_venda])

In [35]:
df_total.to_csv('imoveis.csv', index=False)
